In [1]:
import ROOT
import EventDisplay
import h5py
from sys import exit
import random
from array import array
import numpy as np

Welcome to JupyROOT 6.16/00


In [2]:
f = h5py.File('../Outfile_CellInformation_HomDet_2to5GeV.h5','r')

In [17]:
#topocluster file of Total energy 1nearest 
gT = h5py.File('../DATA_PROCESSING/Outfile_TopoClusterTotal.h5','r')
#topocluster file of Total energy 2nearest
gTA = h5py.File('../DATA_PROCESSING/Outfile_TopoClusterTotalAtl.h5','r')
#topo charged
#gCh = h5py.File('../DATA_PROCESSING/Outfile_TopoClusterTotalAtlCh.h5','r')

In [10]:
def histos(label):

    h = []
    
    h_E_L0= ROOT.TH2D("h_E_L0"+label,"h_E_L0"+label,32,1,33,32,1,33)
    h_E_L1= ROOT.TH2D("h_E_L1"+label,"h_E_L1"+label,64,1,65,64,1,65)
    h_E_L2= ROOT.TH2D("h_E_L2"+label,"h_E_L2"+label,32,1,33,32,1,33)
    h_E_L3= ROOT.TH2D("h_E_L3"+label,"h_E_L3"+label,16,1,17,16,1,17)
    h_E_L4= ROOT.TH2D("h_E_L4"+label,"h_E_L4"+label,16,1,17,16,1,17)
    h_E_L5= ROOT.TH2D("h_E_L5"+label,"h_E_L5"+label,8,1,9,8,1,9)
    
    h_E_L0.Reset()


    h.append(h_E_L0)
    h.append(h_E_L1)
    h.append(h_E_L2)
    h.append(h_E_L3)
    h.append(h_E_L4)
    h.append(h_E_L5)

    return h


In [11]:
def Count_Topo(gio, events, types):
    Num_Topo=np.zeros(events)
    Ne90 = np.zeros(10)
    NN = []
    print("starting loop")
    for ev in range(events):
        if (ev+1)%1000==0: print('still looping',ev)        
            
        
        #sorted energies in topoclusters
        N_Cl = -1
        for lay in range(1,6):
            Topo = gio['TopoClusters'+str(lay)][ev]
            if np.amax(Topo)>N_Cl:
                N_Cl = np.amax(Topo)
            
        NN.append(int(N_Cl))
        
    return NN


In [7]:



#calorimeter structure
import math

X0_ECAL = 0.5 + 14.0
Lambda_int = 16.8 + 79.4


Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
Total_HCAL_Length = 1.5 * Lambda_int + 4.1 * Lambda_int + 1.8 * Lambda_int
Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length #+ 1.0 # -- there is a 1 cm gap between ECAL & HCAL

zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
zpos_ECAL2 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL + 16 * X0_ECAL
zpos_ECAL3 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL

zpos_HCAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL + 1 + 1.5 * Lambda_int/2
zpos_HCAL2 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL + 1 + 1.5 * Lambda_int + 4.1 * Lambda_int/2
zpos_HCAL3 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL + 1 + 1.5 * Lambda_int + 4.1 * Lambda_int + 1.8 * Lambda_int/2

layer_struct = {'layer1' : 32, 'layer2' : 64, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}


#find trajectory 
def MakeTruthTrajectory(Theta, Phi, x_indx, y_indx) : 

    x_orig, y_orig = 0., 0.
    z_orig = -1 * (Total_Calo_Length/2 + 250)

    if(x_indx == 5 and y_indx == 5) : x_orig, y_orig   =  20.,  20.
    if(x_indx == 5 and y_indx == -5) : x_orig, y_orig  =  20., -20.
    if(x_indx == -5 and y_indx == 5) : x_orig, y_orig  = -20.,  20.
    if(x_indx == -5 and y_indx == -5) : x_orig, y_orig = -20., -20.

    layer1_x = x_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer1_y = y_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer2_x = x_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer2_y = y_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer3_x = x_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer3_y = y_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer4_x = x_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer4_y = y_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer5_x = x_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer5_y = y_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer6_x = x_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer6_y = y_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    CellIndexLayer = []

    CellIndexLayer.append(  FindCellIndex(layer1_x, layer1_y, 'layer1') )
    CellIndexLayer.append(  FindCellIndex(layer2_x, layer2_y, 'layer2') )
    CellIndexLayer.append(  FindCellIndex(layer3_x, layer3_y, 'layer3') )
    CellIndexLayer.append(  FindCellIndex(layer4_x, layer4_y, 'layer4') )
    CellIndexLayer.append(  FindCellIndex(layer5_x, layer5_y, 'layer5') )
    CellIndexLayer.append(  FindCellIndex(layer6_x, layer6_y, 'layer6') )

    return np.array(CellIndexLayer)

def FindCellIndex(tr_x, tr_y, layer_name): 
    
    # out_image = np.zeros( [layer_struct[layer_name], layer_struct[layer_name]] )
    
    x_idx = int(  (tr_x+125)/250 * layer_struct[layer_name] )
    y_idx = int(  (tr_y+125)/250 * layer_struct[layer_name]  )
    
    return np.array([x_idx, y_idx])




In [25]:

def EventDisplay(EN_F, CL_F, ev, part, n_clus):
    h_E_tot = histos("Etot_ev_"+str(ev))
    i = 0
    
    h_clus= [ histos("Ev_"+str(ev)+'Clust'+str(i)) for i in range(n_clus[ev])]
    #h_E_tot_sup = histos("Etot_ev_"+str(ev))

    h_Traj= histos("Mark_Traj"+str(ev))
    
    layers=[32,64,32,16,16,8]

    C3=f["Trk_X_indx"]
    C4=f["Trk_Y_indx"]
    C2=f["Trk_Phi"]
    C1=f["Trk_Theta"]

    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev])
    #print(Traj_Mark)
    
    for lay in range(1,7):
        #print(lay)
        Tot_En_Distr=EN_F['RealRes_'+str(part)+'Energy_Layer'+str(lay)][ev][0]
        Clust_Cell=CL_F['TopoClusters'+str(lay)][ev][0]
        
        h_Traj[lay-1].SetBinContent(int(Traj_Mark[lay-1][0])+1 , int(Traj_Mark[lay-1][1])+1 ,10)
        
        for X in range(layers[lay-1]):
            for Y in range(layers[lay-1]):
                if Tot_En_Distr[X][Y]>1:
                    #print(X+1,Y+1,Tot_En_Distr[X][Y])
                    h_E_tot[lay-1].SetBinContent(X+1,Y+1, Tot_En_Distr[X][Y])
                else:
                    h_E_tot[lay-1].SetBinContent(X+1,Y+1, 1)
                TopoI=int(Clust_Cell[X][Y])
                if TopoI!=0:
                    h_clus[TopoI-1][lay-1].SetBinContent(X+1,Y+1,1)
                    #h_E_tot_sup[lay-1].SetBinContent(X+1,Y+1, Tot_En_Distr[X][Y])



    layers=[32,64,32,16, 16, 8]
    col=[ 2,  7 ,8, 9, 11, 0, 3]
    can=ROOT.TCanvas( "ev_"+str(ev), "ev_"+str(ev),1600,900);
    can.SetLogz()
    can.Divide(3,2);
    for j in range(6):
        can.cd(j+1)
        ROOT.gPad.SetLogz()
        ROOT.gStyle.SetOptStat(0)
        h_E_tot[j].Draw("colz")

        # print "proton n=",t+1, " in layer:",j,"h_l max = ",h_l.GetMaximum()," max h[0][1]",  h_l[0][1].GetMaximum()                                                                                       
        for cl in range(n_clus[ev]):
            h_clus[cl][j].SetLineColor(cl)
            h_clus[cl][j].SetFillColor(col[cl])
            h_clus[cl][j].SetLineWidth(1)            
            h_clus[cl][j].SetContour(1);
            h_clus[cl][j].SetMarkerStyle(2)
            h_clus[cl][j].SetMarkerColor(cl)
            h_clus[cl][j].SetMarkerSize(0.4)
            #h_clus[cl][j].SetFillColorAlpha(cl, 0.5)
            
            h_clus[cl][j].Draw("cont3 same");
        h_Traj[j].SetMarkerStyle(3)
        h_Traj[j].SetMarkerColor(1)
        h_Traj[j].SetMarkerSize(10)
        h_Traj[j].SetFillColor(1)
        h_Traj[j].SetLineColor(1)
        h_Traj[j].Draw("box same")


    #can.Update();
    can.Print("Event_"+str(ev)+"_Display"+str(part)+" - 1N.png")

In [30]:
N_Ev=len(gT['TopoClusters1'])

N_ClusT=Count_Topo(gT, N_Ev, "Total")
N_ClusTA=Count_Topo(gTA, N_Ev, "Total")


starting loop
still looping 999
starting loop
still looping 999


In [29]:
print(N_ClusT[3])
EventDisplay(f,gT,3,"Total", N_ClusT)

4


Info in <TCanvas::Print>: png file Event_3_DisplayCharged - 1N.png has been created
